In [1]:
using LRESolve
using LinearAlgebra
using BenchmarkTools

┌ Info: Recompiling stale cache file /home/normann/.julia/compiled/v1.2/LRESolve/hQY0m.ji for LRESolve [ee29a0df-cd28-4477-a8c7-b2df1281527b]
└ @ Base loading.jl:1240


# Sims (2001)

In all the following sections, I consider the linearized simple RBC model developed in Dejong and Dave's book *Structural Macroeconometrics* 2nd ed. p. 27

In [2]:
Γ₀ = [ 0. 1. 0. 0.33 -1. -0.33 -1. ; 0. 1.5 0. -0.06 0.5 0.06 -0.08 ; 1. 0. 0. -0.67 0. -0.33 -1. ; 1. -0.77 -0.23 0. 0. 0. 0. ; 0. 0. 0. 0. 0. 1. 0. ; 0. 0. 0. -0.47 -0.53 0. 0. ; 0. 0. 0. 0. 0. 0. 1.]
Γ₁ = zeros(7,7)
Γ₁[1,6] = 0.33
Γ₁[2,2] = 1.5
Γ₁[2,5] = 0.5
Γ₁[5,3] = 0.06
Γ₁[5,6] = 0.94
Γ₁[7,7] = 0.9

C = zeros(7)

Ψ = zeros(7,1)
Ψ[7,1] = 1.

Π = zeros(7,1)
Π[2,1] = 1.

1.0

In [3]:
M0 = ModelSims(Γ₀,Γ₁,C,Ψ,Π)

ModelSims([0.0 1.0 … -0.33 -1.0; 0.0 1.5 … 0.06 -0.08; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 1.0], [0.0 0.0 … 0.33 0.0; 0.0 1.5 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.9], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0; 0.0; … ; 0.0; 1.0], [0.0; 1.0; … ; 0.0; 0.0])

In [4]:
Θ, Θ₀, Θ₁ = solve_sims(M0)

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.2240299757861992; 0.5931373376949598; … ; -4.1021652063444614e-16; 1.0], [1.4467879103845174e-17 2.7621567106117985e-17 … 0.37933755890654874 1.1016269782075794; 1.7816484208971583e-17 -4.1343644983873385e-17 … 0.5146387165901619 0.5338236039254638; … ; -5.605217849443205e-18 -1.7512168071304799e-16 … 0.9399999999999992 -3.350036493533911e-16; 4.530234343789211e-34 2.827225946090631e-17 … -2.3006691679325895e-17 0.9])

In [5]:
@btime solve_sims(M0)

  31.830 μs (156 allocations: 25.98 KiB)


([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.2240299757861992; 0.5931373376949598; … ; -4.1021652063444614e-16; 1.0], [1.4467879103845174e-17 2.7621567106117985e-17 … 0.37933755890654874 1.1016269782075794; 1.7816484208971583e-17 -4.1343644983873385e-17 … 0.5146387165901619 0.5338236039254638; … ; -5.605217849443205e-18 -1.7512168071304799e-16 … 0.9399999999999992 -3.350036493533911e-16; 4.530234343789211e-34 2.827225946090631e-17 … -2.3006691679325895e-17 0.9])

In [6]:
Θ

7-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [7]:
Θ₀

7×1 Array{Float64,2}:
  1.2240299757861992    
  0.5931373376949598    
  3.336148807656871     
  0.3343730981883576    
 -0.2965195399028826    
 -4.1021652063444614e-16
  1.0                   

In [8]:
Θ₁

7×7 Array{Float64,2}:
  1.44679e-17   2.76216e-17   0.0152197    …   0.379338      1.10163    
  1.78165e-17  -4.13436e-17   0.0281182        0.514639      0.533824   
 -2.96857e-18   4.4192e-16   -0.0279621       -0.0736272     3.00253    
 -6.95286e-18   2.62164e-17  -0.00683619       0.10319       0.300936   
 -6.61299e-18  -1.09716e-17   0.00606229      -0.0915085    -0.266868   
 -5.60522e-18  -1.75122e-16   0.06         …   0.94         -3.35004e-16
  4.53023e-34   2.82723e-17  -2.01012e-18     -2.30067e-17   0.9        

# Uhlig (1999)

In [9]:
n = size(Γ₀,1)
ns = 1
m = n-ns

6

In [10]:
F = zeros(m,m)
G = zeros(m,m)
H = zeros(m,m)
L = zeros(m,1)
M = zeros(m,1)
N = zeros(1,1)

F[2,:] = -Γ₀[2,1:m]
G[2,:] = Γ₁[2,1:m]
L[2,1] = -Γ₀[2,n]
M[2,1] = Γ₁[2,n]

G[[1,3,4,5,6],:] = -Γ₀[[1,3,4,5,6],1:m]
H[[1,3,4,5,6],:] = Γ₁[[1,3,4,5,6],1:m]
M[[1,3,4,5,6],:] = -Γ₀[[1,3,4,5,6],n]
N[1,1] = Γ₁[n,n]

0.9

In [11]:
M1 = ModelUhlig(F,G,H,L,M,N)

ModelUhlig([0.0 0.0 … 0.0 0.0; -0.0 -1.5 … -0.5 -0.06; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [-0.0 -1.0 … 1.0 0.33; 0.0 1.5 … 0.5 0.0; … ; -0.0 -0.0 … -0.0 -1.0; -0.0 -0.0 … 0.53 -0.0], [0.0 0.0 … 0.0 0.33; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.94; 0.0 0.0 … 0.0 0.0], [0.0; 0.08; … ; 0.0; 0.0], [1.0; 0.0; … ; -0.0; -0.0], [0.9])

In [12]:
P,Q = solve_uhlig(M1)

([0.0 0.0 … 0.0 0.3793375589065485; 0.0 0.0 … 0.0 0.5146387165901636; … ; 0.0 0.0 … 0.0 -0.09150845588870084; 0.0 0.0 … 0.0 0.9400000000000001], [1.2240299757861983, 0.5931373376949611, 3.336148807656861, 0.3343730981883557, -0.29651953990288143, -0.0])

In [13]:
@btime solve_uhlig(M1)

  29.336 μs (83 allocations: 26.20 KiB)


([0.0 0.0 … 0.0 0.3793375589065485; 0.0 0.0 … 0.0 0.5146387165901636; … ; 0.0 0.0 … 0.0 -0.09150845588870084; 0.0 0.0 … 0.0 0.9400000000000001], [1.2240299757861983, 0.5931373376949611, 3.336148807656861, 0.3343730981883557, -0.29651953990288143, -0.0])

In [14]:
P

6×6 Array{Float64,2}:
 0.0  0.0   0.0152197   0.0  0.0   0.379338 
 0.0  0.0   0.0281182   0.0  0.0   0.514639 
 0.0  0.0  -0.0279621   0.0  0.0  -0.0736272
 0.0  0.0  -0.00683619  0.0  0.0   0.10319  
 0.0  0.0   0.00606229  0.0  0.0  -0.0915085
 0.0  0.0   0.06        0.0  0.0   0.94     

In [15]:
Q

6-element Array{Float64,1}:
  1.2240299757861983 
  0.5931373376949611 
  3.336148807656861  
  0.3343730981883557 
 -0.29651953990288143
 -0.0                